In [ ]:
!git clone https://github.com/happy-ditto/HypEmo.git

Cloning into 'HypEmo'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 216 (delta 24), reused 23 (delta 23), pack-reused 182
Receiving objects: 100% (216/216), 3.57 MiB | 8.71 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [ ]:
import pandas as pd
output = pd.read_csv('/content/gpt_aug_pred.csv')

In [ ]:
test_data = pd.read_csv('/content/HypEmo/data/go_emotion/test.csv')

In [ ]:
test_data.head()

,text,aug_text,label
0,i’m really sorry about your situation :( altho...,"i'm very sorry about your situation, though i ...",25
1,it's wonderful because it's awful. at not with.,it's wonderful because it's scary.,0
2,"kings fan here, good luck to you guys! will be...","good luck, king's fans!",13
3,"i didn't know that, thank you for teaching me ...","i don't know, thanks for teaching me something...",15
4,thank you for asking questions and recognizing...,and i admit that there are things you may not ...,15


In [ ]:
output = pd.concat([test_data[['text', 'aug_text', 'label']], output], axis=1)

# Reset the index if needed
output.reset_index(drop=True, inplace=True)

In [ ]:
output.head()

,text,aug_text,label,Index,Predicted_Label
0,i’m really sorry about your situation :( altho...,"i'm very sorry about your situation, though i ...",25,0,24
1,it's wonderful because it's awful. at not with.,it's wonderful because it's scary.,0,1,0
2,"kings fan here, good luck to you guys! will be...","good luck, king's fans!",13,2,20
3,"i didn't know that, thank you for teaching me ...","i don't know, thanks for teaching me something...",15,3,15
4,thank you for asking questions and recognizing...,and i admit that there are things you may not ...,15,4,15


## Error Analysis

In [ ]:
output.columns

Index(['text', 'aug_text', 'label', 'Index', 'Predicted_Label'], dtype='object')

In [ ]:
output.to_csv('hypmo_output_analysis.csv')

In [ ]:
label2idx = {'admiration': 0, 'amusement': 1, 'anger': 2,
      'annoyance': 3, 'approval': 4, 'caring': 5,
      'confusion': 6, 'curiosity': 7, 'desire': 8,
      'disappointment': 9, 'disapproval': 10, 'disgust': 11,
      'embarrassment': 12, 'excitement': 13, 'fear': 14,
      'gratitude': 15, 'grief': 16, 'joy': 17,
      'love': 18, 'nervousness': 19, 'optimism': 20,
      'pride': 21, 'realization': 22, 'relief': 23,
      'remorse': 24, 'sadness': 25, 'surprise': 26}
idx2label = {}
for key, val in label2idx.items():
  idx2label[val] = key
output['predict_label'] = output['Predicted_Label'].replace(idx2label)
output['true_label'] = output['label'].replace(idx2label)

In [ ]:
output.head()

,text,aug_text,label,Index,Predicted_Label,predict_label,true_label
0,i’m really sorry about your situation :( altho...,"i'm very sorry about your situation, though i ...",25,0,24,remorse,sadness
1,it's wonderful because it's awful. at not with.,it's wonderful because it's scary.,0,1,0,admiration,admiration
2,"kings fan here, good luck to you guys! will be...","good luck, king's fans!",13,2,20,optimism,excitement
3,"i didn't know that, thank you for teaching me ...","i don't know, thanks for teaching me something...",15,3,15,gratitude,gratitude
4,thank you for asking questions and recognizing...,and i admit that there are things you may not ...,15,4,15,gratitude,gratitude


### Analyze Using Goemotions Taxonomy

In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score
labels = label2idx.keys()  # Get unique labels

f1_scores = []
recall_scores = []
precision_scores = []

for label in labels:
  true_labels = (output['true_label'] == label).astype(int)
  predicted_labels = (output['predict_label'] == label).astype(int)

  f1 = f1_score(true_labels, predicted_labels)
  recall = recall_score(true_labels, predicted_labels)
  precision = precision_score(true_labels, predicted_labels)

  f1_scores.append(f1)
  recall_scores.append(recall)
  precision_scores.append(precision)

In [ ]:
output[output['true_label'] == 'grief']

,text,aug_text,label,Index,Predicted_Label,predict_label,true_label
419,[name] death is just so..... senseless. why? w...,why why why,16,419,7,curiosity,grief
1676,rip the guy from psych,rip that guy out of his mind.,16,1676,2,anger,grief


In [ ]:
result_df = pd.DataFrame({'Label': labels, 'F1_Score': f1_scores, 'Recall': recall_scores, 'Precision': precision_scores})

In [ ]:
result_df

,Label,F1_Score,Recall,Precision
0,admiration,0.744186,0.781609,0.710183
1,amusement,0.850394,0.870968,0.830769
2,anger,0.528455,0.496183,0.565217
3,annoyance,0.393352,0.365979,0.425150
4,approval,0.552419,0.580508,0.526923
5,caring,0.476190,0.406977,0.573770
6,confusion,0.452261,0.463918,0.441176
7,curiosity,0.673522,0.744318,0.615023
8,desire,0.551020,0.482143,0.642857
9,disappointment,0.350649,0.306818,0.409091


### Analyze Using Ekman's Grouping Method

In [ ]:
senti2label = {
"anger": ["anger", "annoyance", "disapproval"],
"disgust": ["disgust"],
"fear": ["fear", "nervousness"],
"joy": ["joy", "amusement", "approval", "excitement", "gratitude",  "love", "optimism", "relief", "pride", "admiration", "desire", "caring"],
"sadness": ["sadness", "disappointment", "embarrassment", "grief",  "remorse"],
"surprise": ["surprise", "realization", "confusion", "curiosity"]
}
label2senti = {}
for key, vals in senti2label.items():
  for val in vals:
    label2senti[val] = key

output['predict_senti'] = output['predict_label'].replace(label2senti)
output['true_senti'] = output['true_label'].replace(label2senti)

In [ ]:
labels = senti2label.keys()  # Get unique labels

f1_scores = []
recall_scores = []
precision_scores = []

for label in labels:
  true_labels = (output['true_senti'] == label).astype(int)
  predicted_labels = (output['predict_senti'] == label).astype(int)

  f1 = f1_score(true_labels, predicted_labels)
  recall = recall_score(true_labels, predicted_labels)
  precision = precision_score(true_labels, predicted_labels)

  f1_scores.append(f1)
  recall_scores.append(recall)
  precision_scores.append(precision)

In [ ]:

result_df = pd.DataFrame({'Label': labels, 'F1_Score': f1_scores, 'Recall': recall_scores, 'Precision': precision_scores})
result_df

,Label,F1_Score,Recall,Precision
0,anger,0.681954,0.657692,0.708075
1,disgust,0.590604,0.578947,0.602740
2,fear,0.745098,0.740260,0.750000
3,joy,0.902476,0.920774,0.884892
4,sadness,0.647303,0.602317,0.699552
5,surprise,0.716484,0.726058,0.707158


## Analyze Using Sentiment Taxomony

In [ ]:
group2label = {
"positive": ["amusement", "excitement", "joy", "love", "desire", "optimism", "caring", "pride", "admiration", "gratitude", "relief", "approval"],
"negative": ["fear", "nervousness", "remorse", "embarrassment", "disappointment", "sadness", "grief", "disgust", "anger", "annoyance", "disapproval"],
"ambiguous": ["realization", "surprise", "curiosity", "confusion"]
}

In [ ]:
label2group = {}
for key, vals in group2label.items():
  for val in vals:
    label2group[val] = key

In [ ]:
print(label2group)

{'amusement': 'positive', 'excitement': 'positive', 'joy': 'positive', 'love': 'positive', 'desire': 'positive', 'optimism': 'positive', 'caring': 'positive', 'pride': 'positive', 'admiration': 'positive', 'gratitude': 'positive', 'relief': 'positive', 'approval': 'positive', 'fear': 'negative', 'nervousness': 'negative', 'remorse': 'negative', 'embarrassment': 'negative', 'disappointment': 'negative', 'sadness': 'negative', 'grief': 'negative', 'disgust': 'negative', 'anger': 'negative', 'annoyance': 'negative', 'disapproval': 'negative', 'realization': 'ambiguous', 'surprise': 'ambiguous', 'curiosity': 'ambiguous', 'confusion': 'ambiguous'}


In [ ]:
output['predict_group'] = output['predict_label'].replace(label2group)
output['true_group'] = output['true_label'].replace(label2group)

In [ ]:
labels = group2label.keys()  # Get unique labels

f1_scores = []
recall_scores = []
precision_scores = []

for label in labels:
  true_labels = (output['true_group'] == label).astype(int)
  predicted_labels = (output['predict_group'] == label).astype(int)

  f1 = f1_score(true_labels, predicted_labels)
  recall = recall_score(true_labels, predicted_labels)
  precision = precision_score(true_labels, predicted_labels)

  f1_scores.append(f1)
  recall_scores.append(recall)
  precision_scores.append(precision)

In [ ]:
result_df = pd.DataFrame({'Label': labels, 'F1_Score': f1_scores, 'Recall': recall_scores, 'Precision': precision_scores})
result_df

,Label,F1_Score,Recall,Precision
0,positive,0.902476,0.920774,0.884892
1,negative,0.813654,0.780043,0.850292
2,ambiguous,0.716484,0.726058,0.707158


In [ ]:
result_df = pd.DataFrame({'Label': labels, 'F1_Score': f1_scores, 'Recall': recall_scores})
result_df

,Label,F1_Score,Recall
0,positive,0.883575,0.875858
1,negative,0.796943,0.783262
2,ambiguous,0.665971,0.710468


### Analyze the correlation between training data size and F1-score

In [ ]:
train_data = pd.read_csv('/content/HypEmo/data/go_emotion/train.csv')
train_data['true_label'] = train_data['label'].replace(idx2label)
train_goe_label = train_data.groupby('true_label').size().to_frame(name='counts').reset_index()

In [ ]:
train_data[train_data['true_label'] == 'admiration']

,text,aug_text,label,true_label
5,damn youtube and outrage drama is super lucrat...,and the angry play is a super-profit re-examin...,0,admiration
18,famous for his 3-4 defense,he's famous for his 3-4 defense.,0,admiration
28,twilight... still a better love story than the...,dawn... is still a better love story than the ...,0,admiration
30,what a wonderful world,what a wonderful world.,0,admiration
44,you just make her sound awesome.,you just make her sound awesome.,0,admiration
...,...,...,...,...
23447,kirkland liquor is amazing. i can get a cheap ...,"i could buy a cheap bottle of vodka or scotch,...",0,admiration
23468,wow nice way to live in an echo chamber.,wow. that's a good way to live in the echo room.,0,admiration
23470,"even when it's bad, it's still pretty good","even if it's bad, it's not bad.",0,admiration
23471,this is my favorite reddit conversation i’ve s...,this is my favorite redacted conversation.,0,admiration


In [ ]:
labels = label2idx.keys()
f1_scores = []
recall_scores = []

for label in labels:
  true_labels = (output['true_label'] == label).astype(int)
  predicted_labels = (output['predict_label'] == label).astype(int)

  f1 = f1_score(true_labels, predicted_labels)
  recall = recall_score(true_labels, predicted_labels)

  f1_scores.append(f1)
  recall_scores.append(recall)
result_df = pd.DataFrame({'Label': labels, 'F1_Score': f1_scores, 'Recall': recall_scores})

In [ ]:
result = pd.merge(train_goe_label, result_df, left_on='true_label', right_on='Label')

In [ ]:
result

,true_label,counts,Label,F1_Score,Recall
0,admiration,2710,admiration,0.732782,0.764368
1,amusement,1652,amusement,0.833787,0.822581
2,anger,1025,anger,0.514768,0.465649
3,annoyance,1451,annoyance,0.354312,0.391753
4,approval,1873,approval,0.470588,0.440678
5,caring,649,caring,0.477778,0.500000
6,confusion,858,confusion,0.392523,0.432990
7,curiosity,1389,curiosity,0.640212,0.687500
8,desire,389,desire,0.465116,0.357143
9,disappointment,709,disappointment,0.359551,0.363636
